In [ ]:
#title Import

!pip install -q jmetalpy



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.8/205.8 kB 2.7 MB/s eta 0:00:00


# Imports

In [ ]:
import math
import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import os

from decimal import Decimal
import re


from google.colab import drive
drive.mount('/gdrive')

### SMPSO E NSGA-2


from jmetal.util.observer import ProgressBarObserver, WriteFrontToFileObserver
from jmetal.util.archive import CrowdingDistanceArchive
from jmetal.algorithm.multiobjective.smpso import SMPSO, DynamicSMPSO
from jmetal.algorithm.multiobjective.omopso import OMOPSO
from jmetal.operator.mutation import NonUniformMutation
from jmetal.operator import UniformMutation
from jmetal.core.problem import OnTheFlyFloatProblem, FloatProblem, IntegerProblem
from jmetal.operator import SBXCrossover, PolynomialMutation, BinaryTournamentSelection, IntegerPolynomialMutation
from jmetal.util.solution import print_function_values_to_file, print_variables_to_file
from jmetal.util.termination_criterion import StoppingByEvaluations
from jmetal.lab.visualization import Plot, InteractivePlot


from jmetal.algorithm.multiobjective.nsgaii import NSGAII
#from jmetal.algorithm.multiobjective.nsgaiii import NSGAIII, UniformReferenceDirectionFactory
from jmetal.operator import BinaryTournamentSelection, BitFlipMutation, SPXCrossover
from jmetal.util.comparator import RankingAndCrowdingDistanceComparator, DominanceComparator

from jmetal.core.quality_indicator import HyperVolume, EpsilonIndicator


from jmetal.algorithm.multiobjective.moead import MOEAD
from jmetal.operator import DifferentialEvolutionCrossover
from jmetal.util.aggregative_function import Tschebycheff

from jmetal.algorithm.multiobjective.gde3 import GDE3



Mounted at /gdrive


# Koon

In [ ]:
def calcParcela(m,n,availabilities):

  av_total = 0
  combinations = list(itertools.combinations(range(1,m+1), n))
  #print(combinations)
  for nColumn in range(len(combinations)):
    combination = combinations[nColumn]
    #print(combination)
    av = 1

    for index in range(1,len(availabilities)+1):
      if index in combination:
        #print("index: ", index)
        av = av*availabilities[index-1]
      else:
        av = av*(1-availabilities[index-1])

    av_total = av_total + av
  return av_total


def koon(availabilities, k):

  n = len(availabilities)
  av = 0

  for i in range(n, (k-1), -1):
    result = calcParcela(n,i,availabilities)
    av = av+result


  return av


def availabilityNoIdenticalKooN(availabilities, quantities, k):

  realAvailabilities = []

  i = 0
  for quantity in quantities:
    if quantity > 0:
      for x in range(0,quantity):
        realAvailabilities.append(availabilities[i])
    i = i+1

  av = koon(realAvailabilities, k)
  return av


#a = calcParcela(2, 2, (0.99998,0.9995))
#print(a)
#print('----')
#b = calcParcela(2, 1, (0.99998,0.9995))
#print(b)




#a = koon((0.99998,0.9995),2)
#print(a)
#print('----')
#b = koon((0.99998,0.9995),1)
#print(b)

# Optimization

In [ ]:
def getQuantities(lines):

  quant = []
  #j = 1

  for i in range(0, len(lines)):

    if '#' in lines[i]:
      x = re.sub("#", "", lines[i])
      quant.append(int(x))
      #j = j+1


  return quant


def getStringComponent(lines, stQuant):

  s = 1
  cpuData = lines[s:(1+stQuant[0])]
  memData = lines[(s+1+stQuant[0]):(s+1+stQuant[0]+stQuant[1])]
  netData = lines[(s+2+stQuant[0]+stQuant[1]):(s+2+stQuant[0]+stQuant[1]+stQuant[2])]
  hdData = lines[(s+3+stQuant[0]+stQuant[1]+stQuant[2]):(4+stQuant[0]+stQuant[1]+stQuant[2]+stQuant[3])]

  custoMinimo = float(lines[len(lines)-3].split(',')[0])
  custoSugerido = float(lines[len(lines)-2])
  requirements = lines[len(lines)-1]

  req = requirements.split(",")
  requirements = list(req)

  requirements = [int(i) for i in requirements]

  data = [cpuData, memData, netData, hdData]

  return data, custoMinimo, custoSugerido, requirements

def fillDataFrameComponent(component):

  dfComponent = {'ID': [], 'quant': [], 'mttf': [], 'mttr': [],'custo': [], 'dispon': []}

  for i in range(len(component)):
    string = component[i].split(",")
    intValues = []
    for val in range(1,len(string)):
      intValues.append(float(string[val]))

    #print(intValues)
    dfComponent['ID'].append(string[0])
    dfComponent['quant'].append(intValues[0])
    dfComponent['mttf'].append(intValues[1])
    dfComponent['mttr'].append(intValues[2])
    dfComponent['custo'].append(intValues[3])
    dfComponent['dispon'].append(intValues[1]/(intValues[1]+intValues[2]))

  df = pd.DataFrame(data=dfComponent, dtype='float')
  return df

def vectorSlicer(vec):

  vecCPU = vec[0:stQuant[1]]
  vecMEM = vec[(stQuant[1]):(stQuant[1]+stQuant[2])]
  vecNET = vec[(stQuant[1]+stQuant[2]):(stQuant[1]+stQuant[2]+stQuant[3])]
  vecHD = vec[(stQuant[1]+stQuant[2]+stQuant[3]):len(vec)]

  return [vecCPU, vecMEM, vecNET, vecHD]

def custoSubType(custos, quantities):

  custoSubType = 0

  for i in range(len(quantities)):
    custoSubType = custoSubType + (quantities[i]*custos[i])

  return custoSubType


def custo(cpuSet, memSet, netSet, hdSet):

  cpuCusto = custoSubType(cpu['custo'],cpuSet)
  memCusto = custoSubType(mem['custo'], memSet)
  netCusto = custoSubType(net['custo'], netSet)
  hdCusto = custoSubType(hd['custo'], hdSet)

  custo = cpuCusto + memCusto + netCusto + hdCusto
  return custo

def availability(cpuSet, memSet, netSet, hdSet):

  cpuAv = availabilityNoIdenticalKooN(cpu['dispon'],cpuSet, requirements[0])
  memAv = availabilityNoIdenticalKooN(mem['dispon'], memSet, requirements[1])
  netAv = availabilityNoIdenticalKooN(net['dispon'], netSet, requirements[2])
  hdAv = availabilityNoIdenticalKooN(hd['dispon'], hdSet, requirements[3])

  av = cpuAv*memAv*netAv*hdAv

  return(av)


def sort(arrayC, disp, var):
  for i in range(0, len(arrayC)):
    p = i
    current_element = arrayC[p]
    c_elem_d = disp[p]
    c_elem_v = var[p]

    while p > 0 and arrayC[p - 1] > current_element:
      arrayC[p] = arrayC[p - 1]
      disp[p] = disp[p - 1]
      var[p] = var[p - 1]
      p -= 1

    arrayC[p] = current_element
    disp[p] = c_elem_d
    var[p] = c_elem_v

  return arrayC, disp, var

In [ ]:
def function1(y):
  x = [int(round(i)) for i in y]
  typeList = vectorSlicer(x)


  if sum(typeList[0]) < requirements[0] or sum(typeList[1]) < requirements[1] or sum(typeList[2]) < requirements[2] or sum(typeList[3]) < requirements[3]:
    return 0

  av = availability(typeList[0],typeList[1],typeList[2],typeList[3])
  return -av


def function2(y):
  x = [int(round(i)) for i in y]
  typeList = vectorSlicer(x)


  if sum(typeList[0]) < requirements[0] or sum(typeList[1]) < requirements[1] or sum(typeList[2]) < requirements[2] or sum(typeList[3]) < requirements[3]:
    return 10000

  custoTotal = custo(typeList[0],typeList[1],typeList[2],typeList[3])

  return custoTotal


def constraint_function(y):

  custoTotal = function2(y)
  return custoSugerido - custoTotal





In [ ]:
def saveResults(algorithm_name, front, path, iter):

  print_function_values_to_file(front, path+algorithm_name+"-PARETO-"+str(iter)+".txt")
  print_variables_to_file(front, path+algorithm_name+"-"+"VAR"+"-"+str(iter)+".txt")

# SMPSO || NSGA-2

In [ ]:
path = "/scenarios/heterogeneity/"
path2 = "/scenarios/variability/"

arq = 'configuration-ISCC-4'
arq2 = 'configuration-SMC-4'
name = arq2
pathUsado = path2
f = open(pathUsado+name+'.txt', 'r')

lines = []
for line in f:
  lines.append(line)

stQuant = getQuantities(lines)
component, custoMinimo, custoSugerido, requirements = getStringComponent(lines, stQuant)
cpu = fillDataFrameComponent (component[0])
mem = fillDataFrameComponent(component[1])
net = fillDataFrameComponent(component[2])
hd = fillDataFrameComponent(component[3])

quantVariaveis = len(cpu) + len(mem) + len(net) + len(hd)


print(cpu)
print(mem)
print(net)
print(hd)
print(custoSugerido)
print(requirements)

           ID  quant           mttf  mttr  custo    dispon
0  Processor0    4.0  292000.000000   6.0  370.0  0.999979
1  Processor1    4.0  341075.688063   6.0  432.0  0.999982
2  Processor2    4.0  374223.246016   6.0  474.0  0.999984
3  Processor3    4.0  423190.865425   6.0  536.0  0.999986
4  Processor4    4.0  465636.976351   6.0  590.0  0.999987
5  Processor5    4.0  491786.896560   6.0  623.0  0.999988
6  Processor6    4.0  557551.155518   6.0  706.0  0.999989
7  Processor7    4.0  584221.999762   6.0  740.0  0.999990
        ID  quant           mttf  mttr  custo    dispon
0  Memory0    4.0  480000.000000   2.5  130.0  0.999995
1  Memory1    4.0  560672.363940   2.5  151.0  0.999996
2  Memory2    4.0  615161.500300   2.5  166.0  0.999996
3  Memory3    4.0  695656.217137   2.5  188.0  0.999996
4  Memory4    4.0  765430.646057   2.5  207.0  0.999997
5  Memory5    4.0  808416.816263   2.5  218.0  0.999997
6  Memory6    4.0  916522.447426   2.5  248.0  0.999997
7  Memory7    4.0  96

<ipython-input-4-c25d5a039677>:56: FutureWarning:

Could not cast to float64, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised.

<ipython-input-4-c25d5a039677>:56: FutureWarning:

Could not cast to float64, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised.

<ipython-input-4-c25d5a039677>:56: FutureWarning:

Could not cast to float64, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised.

<ipython-input-4-c25d5a039677>:56: FutureWarning:

Could not cast to float64, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame',

In [ ]:
drive.mount("/gdrive")
if __name__ == '__main__':


    problem = OnTheFlyFloatProblem() \
        .set_name(name) \
        .add_function(function1) \
        .add_function(function2) \
        .add_constraint(constraint_function)

    for i in range(len(cpu)):
      problem.add_variable(0, int(cpu['quant'][i]))

    for i in range(len(mem)):
      problem.add_variable(0, int(mem['quant'][i]))

    for i in range(len(net)):
      problem.add_variable(0, int(net['quant'][i]))

    for i in range(len(hd)):
      problem.add_variable(0, int(hd['quant'][i]))

    print(problem.number_of_variables)


    max_evaluations = 25000


    # algorithm = NSGAII(
    #     problem=problem,
    #     population_size=100,
    #     offspring_population_size=100,
    #     mutation=PolynomialMutation(probability=1.0 / problem.number_of_variables, distribution_index=20),
    #     crossover=SBXCrossover(probability=1.0, distribution_index=20),
    #     selection=BinaryTournamentSelection(comparator=RankingAndCrowdingDistanceComparator()),
    #     termination_criterion=StoppingByEvaluations(max_evaluations),
    #     dominance_comparator=DominanceComparator()
    # )

    save_path = "/gdrive/My Drive/results/"
    path1 = "GDE3/Variability/Experiment-4/"
    #path0 = "GDE3/Heterogeneity/Experiment-4/"


    if os.path.exists(save_path+path1) == False:
     os.makedirs(save_path+path1)

    algorithm = GDE3(
        problem=problem,
        population_size=100,
        cr=0.5,
        f=0.5,
        termination_criterion=StoppingByEvaluations(max_evaluations))

    front = []
    #algorithm.observable.register(observer=WriteFrontToFileObserver(output_directory=pathUsado+algorithm.get_name()+'/HV.'+ problem.get_name()))

    #for i in range(4,6):
    algorithm.observable.register(observer=ProgressBarObserver(max_evaluations))
    algorithm.run()

    front = algorithm.get_result()
    name = algorithm.get_name()
    saveResults(name, front, save_path+path1, "8")


#     # Save results to file
#     if os.path.exists(pathUsado+algorithm.get_name()) == False:
#       os.makedirs(pathUsado+algorithm.get_name())
#     print_function_values_to_file(front, pathUsado+algorithm.get_name()+'/FUN.'+ problem.get_name()+".txt")
#     print_variables_to_file(front, pathUsado+algorithm.get_name()+'/VAR.'+ problem.get_name()+".txt")

#     print('Algorithm (continuous problem): ' + algorithm.get_name())
#     print('Problem: ' + problem.get_name())
#     print('Computing time: ' + str(algorithm.total_computing_time))

In [ ]:
from jmetal.util.solution import get_non_dominated_solutions, print_function_values_to_file, print_variables_to_file

front_non_dominated = get_non_dominated_solutions(algorithm.get_result())

# save to files
print_function_values_to_file(front_non_dominated, save_path+path0+'FUN.GDE3.NONDOMINATED')


# Gráficos

In [ ]:
f = open(pathUsado+algorithm.get_name()+'/FUN.'+ problem.get_name()+".txt", 'r')
f1 = open(pathUsado+algorithm.get_name()+'/VAR.'+ problem.get_name()+".txt", 'r')

dispo = []
custo = []
for line in f:
  l = line.split(" ")
  dispo.append(Decimal(l[0]))
  custo.append(float(l[1]))

f.close()
var = []

#for v in f1:
#  var.append(v)

for v in f1:
  va = v.split(" ")
  variab = ""
  for i in range(len(va)-1):
    variab = variab + str(int(round(float(va[i])))) + " "
  variab = variab + "\n"
  var.append(variab)

f1.close()
f1 = open(pathUsado+algorithm.get_name()+'/VAR.'+ problem.get_name()+".txt", 'w')

f1.writelines(var)

f1.close()
custo, dispo, var = sort(custo, dispo, var)


plt.plot( custo, dispo, 'go') # green bolinha
plt.plot( custo, dispo, 'k:', color='orange') # linha pontilha orange


#plt.title(problem.get_name())

plt.grid(True)
plt.xlabel("Custo")
plt.ylabel("Disponibilidade")
plt.show()


In [ ]:
class Teste:
  def __init__(self):
    self.objectives = []

listaHV = []

directory = pathUsado+algorithm.get_name()+'/HV.'+ problem.get_name()

arquivos = os.listdir(directory)


#Ordenar os arquivos

arq = [None]*250

for arquivo in arquivos:

  temp = arquivo.split(".")
  local = int(temp[1])
  arq[local] = arquivo


for arquivo in arq:

  f3 = open(directory+'/'+arquivo)
  front.clear()
  dispo = []
  custo = []

  for line in f3:
    l = line.split(" ")
    dispo.append(float(l[0]))
    custo.append(float(l[1]))



  for i in range(len(dispo)):
    front.append(Teste())
    front[i].objectives.append(dispo[i])
    front[i].objectives.append(custo[i])

#   for a in front:
#     print(a.objectives)
  #print(front[0].objectives)


  hv = HyperVolume(reference_point=[100, 572])
  value = hv.compute(front)
  listaHV.append(value)

f1 = open(pathUsado+algorithm.get_name()+'/hyperV.'+ problem.get_name()+".txt", 'w')

f1.writelines(str(listaHV))

f1.close()

plt.plot(listaHV)
plt.title(problem.get_name())
plt.show()

print(1*custoSugerido)

In [ ]:
listaHVNormalizada = [x/(custoSugerido - custoMinimo) for x in listaHV]

print(listaHVNormalizada)

plt.plot(listaHVNormalizada)
plt.title(problem.get_name())
plt.show()